In [1]:
import pandas as pd
import networkx as nx
from sodapy import Socrata
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import copy 
from scipy.cluster.vq import kmeans2, whiten
import seaborn as sns

from util import createDiGraphK2, getOutliers, get_stats

sns.set_style('darkgrid')

#to supress some annoying warnings
import warnings; warnings.simplefilter('ignore')

Below is some simple analysis of the taxi data set 

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gkne-dk5s", where="pickup_datetime between '2014-04-01T00:00:00' and '2014-04-30T23:59:59'", 
                     limit=1000000)

# Convert to pandas DataFrame
taxi14 = pd.DataFrame.from_records(results)

In [3]:
taxi14

,dropoff_datetime,dropoff_latitude,dropoff_longitude,fare_amount,imp_surcharge,mta_tax,passenger_count,payment_type,pickup_datetime,pickup_latitude,pickup_longitude,rate_code,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,vendor_id
0,2014-04-20T10:40:24.000,40.752662000000001,-73.983723999999995,6,0,0.5,1,CSH,2014-04-20T10:35:56.000,40.739441999999997,-73.987893999999997,1,N,0,0,6.5,1.2,CMT
1,2014-04-24T00:25:23.000,40.766454000000003,-73.956057999999999,4.5,0.5,0.5,1,CRD,2014-04-24T00:23:11.000,40.762450000000001,-73.965839000000003,1,N,1,0,6.5,0.80000000000000004,CMT
2,2014-04-07T15:00:00.000,40.779237000000002,-73.960380000000001,5.5,0,0.5,1,CSH,2014-04-07T14:54:00.000,40.774704999999997,-73.950886999999994,1,NaN,0,0,6,0.62,VTS
3,2014-04-07T16:36:53.000,40.723039999999997,-73.998724999999993,16.5,1,0.5,1,CRD,2014-04-07T16:14:52.000,40.763281999999997,-73.974748000000005,1,Y,3,0,21,3.7000000000000002,CMT
4,2014-04-20T12:12:00.000,40.768701999999998,-73.961984999999999,12.5,0,0.5,2,CRD,2014-04-20T12:00:00.000,40.804720000000003,-73.960142000000005,1,NaN,2,0,15,3.21,VTS
5,2014-04-22T14:43:00.000,40.756171999999999,-73.976849999999999,11,0,0.5,1,CSH,2014-04-22T14:29:00.000,40.740527,-74.001985000000005,1,NaN,0,0,11.5,2.2400000000000002,VTS
6,2014-04-16T14:39:00.000,40.788089999999997,-73.953518000000003,4,0,0.5,6,CSH,2014-04-16T14:36:00.000,40.784886999999998,-73.951620000000005,1,NaN,0,0,4.5,0.55000000000000004,VTS
7,2014-04-28T22:03:00.000,40.756101999999998,-74.002647999999994,5,0.5,0.5,1,CSH,2014-04-28T22:00:00.000,40.749856999999999,-73.991860000000003,1,NaN,0,0,6,0.94999999999999996,VTS
8,2014-04-23T20:03:00.000,40.753525000000003,-73.98751,17,1,0.5,3,CSH,2014-04-23T19:38:00.000,40.794037000000003,-73.972707,1,NaN,0,0,18.5,3.1299999999999999,VTS
9,2014-04-22T15:35:50.000,40.742145000000001,-74.003941999999995,7,0,0.5,1,CRD,2014-04-22T15:28:06.000,40.744681,-73.989090000000004,1,N,3,0,10.5,1.3,CMT


In [4]:
# this is so wonderful 
G14 = createDiGraphK2(taxi14)

finding outliers
removing outliers
clustering with k-means
creating graph now


In [12]:
stat_dict = get_stats(G14)
stat_dict

{'betweenness_centrality': {0: 1.7756942964699194e-05,
  1: 1.7756942964699194e-05,
  2: 1.7756942964699194e-05,
  3: 1.7756942964699194e-05,
  4: 1.7756942964699194e-05,
  5: 1.7756942964699194e-05,
  6: 1.7756942964699194e-05,
  7: 1.7756942964699194e-05,
  8: 1.7756942964699194e-05,
  9: 1.7756942964699194e-05,
  10: 1.7756942964699194e-05,
  11: 1.7756942964699194e-05,
  12: 1.7756942964699194e-05,
  13: 1.7756942964699194e-05,
  14: 1.7756942964699194e-05,
  15: 1.7756942964699194e-05,
  16: 1.7756942964699194e-05,
  17: 1.7756942964699194e-05,
  18: 1.7756942964699194e-05,
  19: 1.7756942964699194e-05,
  20: 1.7756942964699194e-05,
  21: 1.7756942964699194e-05,
  22: 1.7756942964699194e-05,
  23: 1.7756942964699194e-05,
  24: 1.7756942964699194e-05,
  25: 1.7756942964699194e-05,
  26: 1.7756942964699194e-05,
  27: 1.7756942964699194e-05,
  28: 1.7756942964699194e-05,
  29: 1.7756942964699194e-05,
  30: 1.7756942964699194e-05,
  31: 1.7756942964699194e-05,
  32: 1.7756942964699194